In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):////////
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import h5py
from PIL import Image
import os
import tensorflow as tf
import PIL
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import pandas as pd
import numpy as np
import keras.backend as K

In [3]:
# #TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [4]:
base_dir = "/kaggle/input/cassava-leaf-disease-classification"

In [5]:
train_csv_path = os.path.join(base_dir, 'train.csv')

In [6]:
train_df = pd.read_csv(train_csv_path)

In [7]:
len(train_df)

21397

In [8]:
train_df.keys()

Index(['image_id', 'label'], dtype='object')

In [9]:
train_df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [10]:
# y_one_hot = tf.one_hot(train_df['label'], depth=5)

In [11]:
# y_one_hot

In [12]:
# train_df['label_encoded'] = y_one_hot

In [13]:
# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(train_df['label'])
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [14]:
# onehot_encoded

In [15]:
# onehot_enccoded_list = onehot_encoded.tolist()

In [16]:
# onehot_enccoded_list

In [17]:
# train_df['label_encoded'] = onehot_enccoded_list

In [18]:
train_df['label_typecasted'] = train_df['label'].apply(str)

In [19]:
train_df

,image_id,label,label_typecasted
0,1000015157.jpg,0,0
1,1000201771.jpg,3,3
2,100042118.jpg,1,1
3,1000723321.jpg,1,1
4,1000812911.jpg,3,3
...,...,...,...
21392,999068805.jpg,3,3
21393,999329392.jpg,3,3
21394,999474432.jpg,1,1
21395,999616605.jpg,4,4


In [20]:
train_df[train_df['label'] == 0]

,image_id,label,label_typecasted
0,1000015157.jpg,0,0
7,1001320321.jpg,0,0
18,1003888281.jpg,0,0
73,1012426959.jpg,0,0
109,1018973237.jpg,0,0
...,...,...,...
21336,986888785.jpg,0,0
21337,986965803.jpg,0,0
21338,986999751.jpg,0,0
21339,987080644.jpg,0,0


In [21]:
images_directory = os.path.join(base_dir, 'train_images')
images_directory

'/kaggle/input/cassava-leaf-disease-classification/train_images'

In [22]:
# #TPU
# from kaggle_datasets import KaggleDatasets
# GCS_PATH = KaggleDatasets().get_gcs_path()
# GCS_PATH
# images_directory = os.path.join(GCS_PATH, 'train_images')
# train_df['image_id'] = images_directory + '/' + train_df['image_id'].astype(str)

In [23]:
# train_df.loc[0]['image_id']

In [24]:
# image_sample = PIL.Image.open('/kaggle/input/cassava-leaf-disease-classification/train_images/1782983052.jpg')

In [25]:
# image_sample

In [26]:
# image_sample.size

In [27]:
# # TPU
# import re
# import sys
# from google.cloud import storage

# BUCKET = GCS_PATH

# # Create a Cloud Storage client.
# gcs = storage.Client()

# # Get the bucket that the file will be uploaded to.
# bucket = gcs.get_bucket(BUCKET)

# def my_list_bucket(bucket_name, limit=sys.maxsize):
#   a_bucket = gcs.lookup_bucket(bucket_name)
#   bucket_iterator = a_bucket.list_blobs()
#   for resource in bucket_iterator:
#     print(resource.name)
#     limit = limit - 1
#     if limit <= 0:
#       break

# my_list_bucket(BUCKET, limit=100)

In [28]:
# image_sample_3_path = os.path.join(images_directory, '1000812911.jpg')

In [29]:
# image_sample_3 = PIL.Image.open('/kaggle/input/cassava-leaf-disease-classification/train_images/1782983052.jpg')

In [30]:
# image_sample_3

In [31]:
# image_sample_4_path = os.path.join(images_directory, '999998473.jpg')

In [32]:
# image_sample_4 = PIL.Image.open(image_sample_4_path)

In [33]:
# image_sample_4

In [34]:
# image_sample_1_directory = os.path.join(images_directory, '999474432.jpg')

In [35]:
# image_sample_1 = PIL.Image.open(image_sample_1_directory)

In [36]:
# image_sample_1

In [37]:
# image_sample_2_directory = os.path.join(images_directory, '996947690.jpg')

In [38]:
# image_sample_2 = PIL.Image.open(image_sample_2_directory)

In [39]:
# image_sample_2

In [40]:
# images_sample_0 = PIL.Image.open(os.path.join(images_directory, '1012426959.jpg'))

In [41]:
# images_sample_0

In [42]:
# image_sample.size

In [43]:
len(train_df)

21397

In [44]:
def get_custom_function(image):
#     input_image = tf.keras.layers.experimental.preprocessing.Resizing(600, 600)(image)
#     input_image = tf.keras.layers.experimental.preprocessing.CenterCrop(300, 300)(input_image)
#     return input_image
    input_image = tf.image.resize(image, [600,600])
    input_image = tf.image.stateless_random_hue(input_image, 0.3, (1,2))
    input_image = tf.image.stateless_random_contrast(input_image, 0.7, 1.0, (1,2))
    input_image = tf.image.stateless_random_saturation(input_image, 0.5, 1.0, (1, 2))
    return tf.image.central_crop(input_image, central_fraction=0.5)

In [45]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.01,
#     preprocessing_function=tf.keras.layers.experimental.preprocessing.CenterCrop(200, 200),
    preprocessing_function = get_custom_function,
#     zca_whitening=True,
    horizontal_flip = 0.5,
    vertical_flip=0.2,
    rotation_range=30,
    brightness_range=[0.55,1.0]
)

In [46]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory=images_directory,
x_col="image_id",
y_col="label_typecasted",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
# horizontal_flip = 0.5,
# vertical_flip=0.2,
# # rotation_range=30,
# # brightness_range=[0.4,1.0],
# rotation_range=10,
# brightness_range=[0.75,1.0],
target_size=(300,300))

Found 21184 validated image filenames belonging to 5 classes.


In [47]:
len(train_generator)

662

In [48]:
32*662

21184

In [49]:
7*32

224

In [50]:
21184 + 213

21397

In [51]:
# def identity_block(X, f, filters, stage, block):
#     """
#     Implementation of the identity block as defined in Figure 4
    
#     Arguments:
#     X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
#     f -- integer, specifying the shape of the middle CONV's window for the main path
#     filters -- python list of integers, defining the number of filters in the CONV layers of the main path
#     stage -- integer, used to name the layers, depending on their position in the network
#     block -- string/character, used to name the layers, depending on their position in the network
    
#     Returns:
#     X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
#     """
    
#     # defining name basis
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'
    
#     # Retrieve Filters
#     F1, F2, F3 = filters
    
#     # Save the input value. You'll need this later to add back to the main path. 
#     X_shortcut = X
    
#     # First component of main path
#     X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
#     X = Activation('relu')(X)
        
#     # Second component of main path (≈3 lines)
#     X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
#     X = Activation('relu')(X)

#     # Third component of main path (≈2 lines)
#     X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

#     # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
#     X = Add()([X_shortcut, X])
#     X = Activation('relu')(X)
    
    
#     return X

In [52]:
# def convolutional_block(X, f, filters, stage, block, s = 2):
#     """
#     Implementation of the convolutional block as defined in Figure 4
    
#     Arguments:
#     X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
#     f -- integer, specifying the shape of the middle CONV's window for the main path
#     filters -- python list of integers, defining the number of filters in the CONV layers of the main path
#     stage -- integer, used to name the layers, depending on their position in the network
#     block -- string/character, used to name the layers, depending on their position in the network
#     s -- Integer, specifying the stride to be used
    
#     Returns:
#     X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
#     """
    
#     # defining name basis
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'
    
#     # Retrieve Filters
#     F1, F2, F3 = filters
    
#     # Save the input value
#     X_shortcut = X

#     ##### MAIN PATH #####
#     # First component of main path 
#     X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
#     X = Activation('relu')(X)
    
#     # Second component of main path (≈3 lines)
#     X = Conv2D(F2, (f, f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
#     X = Activation('relu')(X)

#     # Third component of main path (≈2 lines)
#     X = Conv2D(F3, (1, 1), strides = (1,1), padding='valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

#     ##### SHORTCUT PATH #### (≈2 lines)
#     X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), padding='valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
#     X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

#     # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
#     X = Add()([X, X_shortcut])
#     X = Activation('relu')(X)
        
#     return X

In [53]:
# def ResNet50(input_shape = (64, 64, 3), classes = 6):
#     """
#     Implementation of the popular ResNet50 the following architecture:
#     CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
#     -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

#     Arguments:
#     input_shape -- shape of the images of the dataset
#     classes -- integer, number of classes

#     Returns:
#     model -- a Model() instance in Keras
#     """
    
#     # Define the input as a tensor with shape input_shape
#     X_input = Input(input_shape)

    
#     # Zero-Padding
#     X = ZeroPadding2D((3, 3))(X_input)
    
#     # Stage 1
#     X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
#     X = Activation('relu')(X)
#     X = MaxPooling2D((3, 3), strides=(2, 2))(X)

#     # Stage 2
#     X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
#     X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
#     X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

#     # Stage 3 (≈4 lines)
#     X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

#     # Stage 4 (≈6 lines)
#     X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

#     # Stage 5 (≈3 lines)
#     X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
#     X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
#     X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

#     # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
#     X = AveragePooling2D(pool_size=(2, 2), strides=None, padding="valid", name="avg_pool")(X)
    
#     # output layer
#     X = Flatten()(X)
#     X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
#     # Create model
#     model = Model(inputs = X_input, outputs = X, name='ResNet50')

#     return model

In [54]:
# model = ResNet50(input_shape = (400, 300, 3), classes=5)

In [55]:
# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
# class_weight = {0: 1.0,
#                 1: 1.0,
#                 2: 1.0,
#                 3: 0.25,
#                 4: 1.15}

In [57]:
# model.fit(train_generator, epochs=2, class_weight=class_weight)

# model.fit(train_generator, epochs=5)

In [58]:
val_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory=images_directory,
x_col="image_id",
y_col="label_typecasted",
subset="validation",
batch_size=32,
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(300,300))

Found 213 validated image filenames belonging to 5 classes.


In [59]:
len(val_generator)

7

In [60]:
# center_crop_augmentation = tf.keras.layers.experimental.preprocessing.CenterCrop(
#     height=1, width=1
# )

In [61]:
# datagen_cropped = tf.keras.preprocessing.image.ImageDataGenerator(
#     validation_split=0.01,
#     preprocessing_function=center_crop_augmentation
# )

In [62]:
# train_generator_center_cropped=datagen_cropped.flow_from_dataframe(
# dataframe=train_df,
# directory=images_directory,
# x_col="image_id",
# y_col="label_typecasted",
# subset="training",
# batch_size=32,
# seed=42,
# shuffle=True,
# class_mode="categorical",
# horizontal_flip = 0.5,
# vertical_flip=0.2,
# # rotation_range=30,
# # brightness_range=[0.4,1.0],
# rotation_range=10,
# brightness_range=[0.75,1.0],
# # target_size=(300, 300)
# )

In [63]:
# for batch, image in train_generator_center_cropped:
#     print(batch.shape, image.shape)
#     break

In [64]:
# preds=model.evaluate(val_generator)

In [65]:
# from kaggle_datasets import KaggleDatasets
# import re

In [66]:
# GCS_PATH = KaggleDatasets().get_gcs_path()

In [67]:
# def count_data_items(filenames):
#     n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
#     return np.sum(n)

In [68]:
# def get_test_dataset(ordered=False):
#     dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTOTUNE)
#     return dataset

In [69]:
# def load_dataset(filenames, labeled=True, ordered=False):
#     ignore_order = tf.data.Options()
#     if not ordered:
#         ignore_order.experimental_deterministic = False # disable order, increase speed
#     dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
#     dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
#     dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
#     return dataset

In [70]:
# from functools import partial

In [71]:
# def read_tfrecord(example, labeled):
#     tfrecord_format = {
#         "image": tf.io.FixedLenFeature([], tf.string),
#         "target": tf.io.FixedLenFeature([], tf.int64)
#     } if labeled else {
#         "image": tf.io.FixedLenFeature([], tf.string),
#         "image_name": tf.io.FixedLenFeature([], tf.string)
#     }
#     example = tf.io.parse_single_example(example, tfrecord_format)
#     image = decode_image(example['image'])
#     if labeled:
#         label = tf.cast(example['target'], tf.int32)
#         return image, label
#     idnum = example['image_name']
#     return image, idnum

In [72]:
# def decode_image(image):
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.cast(image, tf.float32) / 255.0
#     image = tf.reshape(image, [*IMAGE_SIZE, 3])
#     return image

In [73]:
# # this code will convert our test image data to a float32 
# def to_float32(image, label):
#     return tf.cast(image, tf.float32), label

In [74]:
# # BATCH_SIZE = 16 * strategy.num_replicas_in_sync
# BATCH_SIZE = 32
# IMAGE_SIZE = [400, 300]
# TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')
# print(TEST_FILENAMES)
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# testing_dataset = get_test_dataset()
# testing_dataset = testing_dataset.unbatch().batch(32)
# NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
# print(NUM_TEST_IMAGES)

In [75]:
# test_ds = get_test_dataset(ordered=True) 
# test_ds = test_ds.map(to_float32)
# print('Computing predictions...')
# test_images_ds = testing_dataset
# test_images_ds = test_ds.map(lambda image, idnum: image)
# print(test_images_ds)
# probabilities = model.predict(test_images_ds)
# # predictions = np.argmax(probabilities, axis=-1)
# # print(predictions)

In [76]:
# print('Generating submission.csv file...')
# test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
# test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
# np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
# !head submission.csv

In [77]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

In [78]:
import pandas as pd

In [79]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.optimizers import SGD

In [80]:
IMG_HEIGHT=300
IMG_WIDTH=300

In [81]:
# resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# resnet.summary()

In [82]:
# output = resnet.layers[-1].output

In [83]:
# resnet.trainable = True

In [84]:
# set_trainable = False
# for layer in resnet.layers:
#     if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False
# layers = [(layer, layer.name, layer.trainable) for layer in resnet.layers]
# pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [85]:
# output

In [86]:
# resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))

In [87]:
!mkdir /kaggle/working/training_models

In [88]:
# # Config 1
# resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# model = tf.keras.Sequential()
# model.add(resnet_base)
# model.add(BatchNormalization(axis=-1))
# model.add(GlobalAveragePooling2D())
# model.add(Dropout(0.5))
# model.add(Dense(5, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [89]:
# # Model Generation - Resnet
# resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# x = resnet_base.output #<KerasTensor: shape=(None, 13, 10, 2048) dtype=float32 (created by layer 'conv5_block3_out')>
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(5, activation='softmax')(x)
# model = Model(inputs=resnet_base.input, outputs=predictions)

# # model.layers
# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in resnet_base.layers:
#     layer.trainable = False

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# checkpoint_1 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_1{epoch:04d}.h5', period=1)
# model.fit(train_generator, epochs=15, callbacks=[checkpoint_1])

# for layer in model.layers[-13:]:
#     layer.trainable = True
# for layer in model.layers[:-13]:
#     layer.trainable = False

# layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
# print("Layers:")
# print(layers)
# checkpoint_2 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_2{epoch:04d}.h5', period=1)
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# # opt = keras.optimizers.Adam(learning_rate=0.001)
# # model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_generator, epochs=15, callbacks=[checkpoint_2])

In [90]:
# # Model Generation - EfficientNet
# # https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
# IMG_SIZE = 300
# NUM_CLASSES = 5
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras import layers
# inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# efficient_net_base_model = EfficientNetB3(weights="/kaggle/input/efficientnetb3-notop/efficientnetb3_notop.h5", include_top=False, drop_connect_rate=0.3, input_shape=(IMG_SIZE,IMG_SIZE,3), input_tensor=inputs)
# # Freeze the pretrained weights
# efficient_net_base_model.trainable = False
# x = layers.GlobalAveragePooling2D(name="avg_pool")(efficient_net_base_model.output)
# x = layers.BatchNormalization()(x)
# top_dropout_rate = 0.2
# x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
# outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
# model = tf.keras.Model(inputs, outputs, name="EfficientNet")
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# checkpoint_1 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_6{epoch:04d}.h5', period=1)
# # model.layers
# intial_epoch = 9
# model.fit(train_generator, epochs=intial_epoch, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_1])
# for layer in model.layers[-20:]:
#     if not isinstance(layer, layers.BatchNormalization):
#         layer.trainable = True

# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# checkpoint_2 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_7{epoch:04d}.h5', period=1)
# final_epoch = 20
# model.fit(train_generator, epochs=final_epoch, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_2])

In [91]:
IMG_SIZE = 300
NUM_CLASSES = 5
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import layers
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# model = tf.keras.models.load_model('../input/casava-trained-model-efficient-net-15-epochs/cassava_trained_model_efficient_net_15_epochs.h5')
# model = tf.keras.models.load_model('../input/cassava-29-epochs-70020/cassava_trained_model_efficient_net_29_epochs.h5')
# model = tf.keras.models.load_model('../input/cassava-additional-augmentation/model_80015.h5')
model = tf.keras.models.load_model('../input/efficientnet-augmneted/model_1230011.h5')
for layer in model.layers[0:-42]:
    layer.trainable = False
for layer in model.layers[-42:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
checkpoint_3 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_345{epoch:04d}.h5', period=1)
final_epoch = 6
model.fit(train_generator, epochs=final_epoch, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_3])

Epoch 1/6
662/662 [==============================] - 853s 1s/step - loss: 0.5476 - accuracy: 0.8039 - val_loss: 0.6426 - val_accuracy: 0.7934
Epoch 2/6
662/662 [==============================] - 744s 1s/step - loss: 0.4752 - accuracy: 0.8307 - val_loss: 0.5736 - val_accuracy: 0.7887
Epoch 3/6
662/662 [==============================] - 744s 1s/step - loss: 0.4616 - accuracy: 0.8388 - val_loss: 0.5755 - val_accuracy: 0.7934
Epoch 4/6
662/662 [==============================] - 741s 1s/step - loss: 0.4075 - accuracy: 0.8519 - val_loss: 0.5164 - val_accuracy: 0.8357
Epoch 5/6
662/662 [==============================] - 752s 1s/step - loss: 0.4004 - accuracy: 0.8554 - val_loss: 0.5720 - val_accuracy: 0.8216
Epoch 6/6
662/662 [==============================] - 739s 1s/step - loss: 0.3814 - accuracy: 0.8633 - val_loss: 0.5682 - val_accuracy: 0.8075


In [92]:
# IMG_SIZE = 300
# NUM_CLASSES = 5
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras import layers
# inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# # model = tf.keras.models.load_model('../input/casava-trained-model-efficient-net-15-epochs/cassava_trained_model_efficient_net_15_epochs.h5')
# # model = tf.keras.models.load_model('../input/cassava-trained-model-49-epochs-823/model_80020.h5')
# model = tf.keras.models.load_model('../input/efficientnet-latest/model_90009.h5')
# # model.layers[-65:]
# for layer in model.layers[0:-65]:
#     layer.trainable = False
# for layer in model.layers[-65:]:
#     if not isinstance(layer, layers.BatchNormalization):
#         layer.trainable = True
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# checkpoint_4 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_9{epoch:04d}.h5', period=1)
# final_epoch = 20
# model.fit(train_generator, epochs=final_epoch, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_4])

In [93]:
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_69_epochs.h5')
# model.save('/kaggle/working/cassava_trained_model_efficient_net_49_epochs.h5')
# model.save('cassava_trained_model_efficient_net_69_epochs.h5')
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_30_epochs.h5')
# model.save('/kaggle/working/cassava_trained_model_efficient_net_30_epochs.h5')
# model.save('cassava_trained_model_efficient_net_30_epochs.h5')
model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_31_epochs.h5')
model.save('/kaggle/working/cassava_trained_model_efficient_net_31_epochs.h5')
model.save('cassava_trained_model_efficient_net_31_epochs.h5')

In [94]:
# # # Model Generation - Resnet
# IMG_HEIGHT=300
# IMG_WIDTH=300
# resnet_base = tf.keras.applications.ResNet152(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# x = resnet_base.output #<KerasTensor: shape=(None, 13, 10, 2048) dtype=float32 (created by layer 'conv5_block3_out')>
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(5, activation='softmax')(x)
# model = Model(inputs=resnet_base.input, outputs=predictions)
# # model.layers[-65:]

# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in resnet_base.layers:
#     layer.trainable = False

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# checkpoint_5 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_010{epoch:04d}.h5', period=1)
# model.fit(train_generator, epochs=15, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_5])

# for layer in model.layers[0:-65]:
#     layer.trainable = False
# for layer in model.layers[-65:]:
#     if not isinstance(layer, layers.BatchNormalization):
#         layer.trainable = True

# # layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
# # print("Layers:")
# # print(layers)
# checkpoint_6 = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/training_models/model_020{epoch:04d}.h5', period=1)
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# # opt = keras.optimizers.Adam(learning_rate=0.001)
# # model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_generator, epochs=35, validation_data=val_generator, steps_per_epoch=len(train_generator), callbacks=[checkpoint_6])

In [95]:
# Model Saving
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net.h5')
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_25_epochs.h5')
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_29_epochs.h5')
# model.save('/kaggle/working/training_models/cassava_trained_model_efficient_net_49_epochs.h5')

In [96]:
# model.save('/kaggle/working/cassava_trained_model_efficient_net.h5')
# model.save('/kaggle/working/cassava_trained_model_efficient_net_25_epochs.h5')
# model.save('/kaggle/working/cassava_trained_model_efficient_net_29_epochs.h5')
# model.save('/kaggle/working/cassava_trained_model_efficient_net_49_epochs.h5')

In [97]:
# model.save('cassava_trained_model_efficient_net.h5')
# model.save('cassava_trained_model_efficient_net_25_epochs.h5')
# model.save('cassava_trained_model_efficient_net_29_epochs.h5')
# model.save('cassava_trained_model_efficient_net_49_epochs.h5')

In [98]:
# model_saved = tf.keras.models.load_model('/kaggle/working/training_models/cassava_trained_model.h5')

In [99]:
# model_saved = tf.keras.models.load_model("../working/training_models/cassava_trained_model.h5")

In [100]:
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net.h5')
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_25_epochs.h5')
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_29_epochs.h5')
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_49_epochs.h5')
# model_saved_resnet = tf.keras.models.load_model('../input/cassava-trained-model-resnet-764/cassava_trained_model.h5')
# model_saved_efficient_net = tf.keras.models.load_model('../input/cassava-trained-model-49-epochs-823/model_80020.h5')
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_69_epochs.h5')
# model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_30_epochs.h5')
model_saved = tf.keras.models.load_model('cassava_trained_model_efficient_net_31_epochs.h5')

In [101]:
preds_saved_model = model_saved.evaluate(val_generator)
# preds_saved_resnet_model = model_saved_resnet.evaluate(val_generator)
# preds_saved_efficient_net_model = model_saved_efficient_net.evaluate(val_generator)

7/7 [==============================] - 10s 1s/step - loss: 0.6063 - accuracy: 0.7934


In [102]:
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# base_model = InceptionV3(weights='imagenet', include_top=False)

# # add a global spatial average pooling layer
# x_inc = base_model.output
# x_inc = GlobalAveragePooling2D()(x_inc)
# # let's add a fully-connected layer
# x_inc = Dense(1024, activation='relu')(x_inc)
# # and a logistic layer -- let's say we have 200 classes
# predictions_inc = Dense(200, activation='softmax')(x_inc)
# model_inc = Model(inputs=base_model.input, outputs=predictions_inc)


In [103]:
# model_inc.layers[249:]

In [104]:
# # TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [105]:
# # TPU
# with tpu_strategy.scope():
#     resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
#     model = tf.keras.Sequential()
#     model.add(resnet_base)
#     model.add(BatchNormalization(axis=-1))
#     model.add(GlobalAveragePooling2D())
#     model.add(Dropout(0.5))
#     model.add(Dense(5, activation='softmax'))
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     model.summary()

In [106]:
# len(resnet_base.layers)

In [107]:
pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

,image_id,label
0,2216849948.jpg,4


In [108]:
test_images = []
for dirname, _, filenames in os.walk('/kaggle/input/cassava-leaf-disease-classification/test_images'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        test_images.append(filename)
# test_images

In [109]:
test_images_csv = '/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv'
test_images_directory = '/kaggle/input/cassava-leaf-disease-classification/test_images'
df_test = pd.read_csv(test_images_csv)

In [110]:
# df_test['image_id']
# df_test['label_typecasted'] = train_df['label'].apply(str)

df_test['image_id'] = test_images
df_test['label'] = None
df_test['label'] = train_df['label'].apply(str)
# print(df_test)

In [111]:
test_path = '/kaggle/input/cassava-leaf-disease-classification/'
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
test_generator=datagen.flow_from_dataframe(
dataframe=df_test,
directory=test_images_directory,
x_col="image_id",
y_col="label",
batch_size=32,
seed=42,
shuffle=False,
color_mode="rgb",
class_mode="categorical",
target_size=(300,300))

Found 1 validated image filenames belonging to 1 classes.


In [112]:
test_generator.reset()
preds=model_saved.predict(test_generator)
# # preds=model.predict(test_generator)
# preds_efficient_net=model_saved_efficient_net.predict(test_generator)
# test_generator.reset()
# preds_resnet=model_saved_resnet.predict(test_generator)

In [113]:
# print(preds_resnet)
# print(preds_efficient_net)
print(preds)

[[0.00336947 0.0032187  0.23115799 0.00103071 0.7612232 ]]


In [114]:
# a = 82.3 / (76.4 + 82.3)
# b = 76.4 / (76.4 + 82.3)

In [115]:
# print(a, b)

In [116]:
# preds_efficient_net = a * preds_efficient_net
# preds_resnet = b * preds_resnet

In [117]:
# preds_efficient_net

In [118]:
# preds_resnet

In [119]:
# total_preds = preds_efficient_net + preds_resnet

In [120]:
# total_preds

In [121]:
# for i, pred in enumerate(total_preds):
#     df_test.loc[i, 'label'] = np.argmax(pred)
for i, pred in enumerate(preds):
    df_test.loc[i, 'label'] = np.argmax(pred)

In [122]:
df_test

,image_id,label
0,2216849948.jpg,4


In [123]:
df_test.to_csv('submission.csv', index=False)
!head submission.csv

image_id,label
2216849948.jpg,4
